# fairseq 
## 체크포인트 경로
/data/koo/datasets/bts/AIHUB/clean/AIHUB_all_/ckpt/checkpoint_best.pt

##bin 파일 경로 
/data/koo/datasets/bts/AIHUB/clean/bin-1.0

# huggingface fsmt 모델
https://huggingface.co/facebook/wmt19-en-de

# 기존 gen 파라미터
python fairseq/fairseq_cli/generate.py datasets/cyd_dataset/bin-cyd  \
--path ./cyd-ckpt/ckpt/checkpoint_best.pt \
--task translation \
--source-lang src --target-lang tgt \
--gen-subset test \
--empty-cache-freq 1000 \
--batch-size 150 \
--remove-bpe sentencepiece > results.txt


# train 인자
GPUNUM=0
DIRNAME=./datasets/cyd_dataset/bin-cyd
FILENAME=./cyd-ckpt

CUDA_VISIBLE_DEVICES=${GPUNUM} nohup fairseq-train \
${DIRNAME} \
—fp16 \
—fp16-init-scale 4096 \
—arch transformer —share-decoder-input-output-embed \
—optimizer adam —adam-betas '(0.9, 0.98)' —clip-norm 0.0 \
—lr 5e-4 —lr-scheduler inverse_sqrt —warmup-updates 4000 \
—dropout 0.3 —weight-decay 0.0001 \
—task translation \
—criterion label_smoothed_cross_entropy —label-smoothing 0.1 \
—max-tokens 4096 \
—update-freq 4 \
—eval-bleu \
—eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
—eval-bleu-detok moses \
—eval-bleu-remove-bpe \
—eval-bleu-print-samples \
—best-checkpoint-metric bleu —maximize-best-checkpoint-metric \
—tensorboard-logdir tblog \
—log-interval 100 \
—max-update 200000 \
—skip-invalid-size-inputs-valid-test \
—no-epoch-checkpoints \
—save-dir ${FILENAME}/ckpt  \
—share-all-embeddings &


In [2]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /mnt/raid6/persuade/miniconda3/envs/117/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /mnt/raid6/persuade/miniconda3/envs/117/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /mnt/raid6/persuade/miniconda3/envs/117/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/mnt/raid6/persuade/miniconda3/envs/117/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/mnt/raid6/persuade/miniconda3/envs/117/lib/libcudart.so'), PosixPath('/mnt/raid6/persuade/miniconda3/envs/117/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [4]:
mname = "facebook/wmt19-en-de"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import json
from transformers.convert_slow_tokenizer import SentencePieceExtractor

vocab_scores = {}
with open('/data/koo/datasets/bts/AIHUB/clean/spm.vocab', 'r') as fp:
    for line in fp.readlines():
        if len(line.split('\t')) != 2:
            raise Exception(f'error while reading spm.vocab! {line}')
        vocab_scores[line.split('\t')[0]] = float(line.split('\t')[1])

vocab, merges = SentencePieceExtractor('/data/koo/datasets/bts/AIHUB/clean/spm.model').extract(vocab_scores)

with open('merges.txt', 'w') as fp:
    fp.write('\n'.join([' '.join(pair) for pair in merges]))
with open('vocab-tgt.json', 'w') as fp:
    json.dump(vocab, fp, ensure_ascii=False, indent=4)
with open('vocab-src.json', 'w') as fp:
    json.dump(vocab, fp, ensure_ascii=False, indent=4)